In [21]:
from azureml.core import Workspace

In [22]:
ws = Workspace.from_config()

In [30]:
from azureml.core import Environment
# env = Environment.get(workspace=ws, name='mag-python-mlflow-basic-cpu')
env = Environment.get(workspace=ws, name='AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu')

In [66]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

cluster_name = 'cpucluster'
cluster_name = 'magazureml02'

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    compute_target.wait_for_completion(show_output=True)
except ComputeTargetException as e:
    print(e)


Running


In [91]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory='.\src',
                      script='train.py',
                      arguments=['--kernel', 'linear', '--penalty', 1.0],
                      compute_target=compute_target,
                      environment=env)

In [92]:
from azureml.core import Experiment

# run = Experiment(ws,'diabetes-experiment').submit(src)
# run.wait_for_completion(show_output=True)

In [93]:
from azureml.data import OutputFileDatasetConfig
from azureml.core.compute import AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline

In [94]:
from azureml.core import Dataset
input_dataset = Dataset.get_by_name(ws, name='diabetes')


In [98]:
from azureml.core import ScriptRunConfig
from azureml.core import Environment

## env created using cli v2 : https://github.com/mutazag/aml/blob/main/estimate_pi/sh_createenv.sh 
env = Environment.get(workspace=ws, name='mag-python-mlflow-basic-cpu')
# env = Environment.get(workspace=ws, name='AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu')

train_runconfig = ScriptRunConfig(source_directory="./src", environment=env)

In [105]:
script_folder = "./src"

train_step = PythonScriptStep(
    name="train sklearn regression",
    source_directory=script_folder,
    script_name='train.py',
    arguments=[
        '--kernel', 'linear', 
        '--penalty', 1.0,
        '--train_dataset', input_dataset.as_named_input('train'),
        '--dataset_download', input_dataset.to_parquet_files().as_named_input('dataset_download').as_download()],
    inputs=[input_dataset.to_parquet_files().as_named_input('input_dataset').as_download()],
    compute_target=compute_target,
    allow_reuse=True,
    runconfig=train_runconfig.run_config

)


In [106]:
pipeline = Pipeline(ws, steps=[train_step])
run = Experiment(ws,'diabetes-experiment').submit(pipeline)

Created step train sklearn regression [e34ba00d][30fab925-9437-4f3b-bceb-6dbfc6da91bf], (This step will run and generate new outputs)
Submitted PipelineRun 61d1d87e-11a7-4e67-a154-126a37603eb8
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/61d1d87e-11a7-4e67-a154-126a37603eb8?wsid=/subscriptions/907c8efc-c2c8-4c49-a4e1-aeb880e10c88/resourcegroups/azureml/workspaces/magazureml&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
